# Tradução dos Códigos de Ocupação O*NET para ocupações do Brasil (COD 2010)

Para aplicar a metodologia de Dingel e Neiman, é necessário realizar a tradução dos códigos de ocupações para ocupações do Brasil.

**Passo-a-passo da tradução dos códigos de ocupação, segundo artigo da FGV:**

1. Primeiro, realizamos uma tradução das 968 ocupações da O*NET para a Standard Occupational Classification de 2010 (SOC-2010), utilizada na pesquisa por domicílios americana (Current Population Survey - CPS);
2. Em seguida, reclassificamos as ocupações da SOC-2010 para a ISCO-08, para então; 
3. traduzirmos para a COD utilizada na PNAD Contínua. 

As reclassificações foram realizadas conforme o diagrama abaixo.

#### Diagrama para tradução dos códigos de ocupação (FGV, 2021)
![Diagrama](../Anexos/diagrama_traducao.png)

Link para base de ocupações da O*NET: [O*NET OnLine](https://www.onetonline.org/find/all)

Link para base de ocupações da SOC-2010: [U.S. BUREAU OF LABOR STATISTICS](https://www.bls.gov/soc/2010/home.htm)

Link para base de ocupações da ISCO-08: [International Labour Organization](https://isco-ilo.netlify.app/en/isco-08/)

Repositório do estudo Work at Home (Dingel e Neiman): [Work at Home](https://github.com/jdingel)

Arquivo CSV com as ocupações teletrabalháveis: [occupations_workathome.csv](https://raw.githubusercontent.com/jdingel/DingelNeiman-workathome/master/occ_onet_scores/output/occupations_workathome.csv)



In [ ]:
#TODO: refazer o código do SOC 2010 para traduzir para código da ISCO-08

In [1]:
#pacotes
import pandas as pd
import numpy as np

In [116]:
#Leitura das bases (tabelas de ocupação e de pessoas)

onet = pd.read_excel('../Anexos/Occupations_ONET.xlsx') #Ocupações O*NET
soc2010 = pd.read_excel('../Anexos/Occupations_SOC-2010.xlsx') #Ocupações SOC-2010
isco08 = pd.read_excel('../Anexos/Occupations_ISCO-08.xlsx') #Ocupações ISCO-08
cod2010 = pd.read_excel('../Anexos/COD 2010.xlsx') #Ocupações COD 2010

onet_teleworkable = pd.read_csv('https://raw.githubusercontent.com/jdingel/DingelNeiman-workathome/master/occ_onet_scores/output/occupations_workathome.csv')

In [152]:
onet_teleworkable

,onetsoccode,title,teleworkable
0,11-1011.00,Chief Executives,1
1,11-1011.03,Chief Sustainability Officers,1
2,11-1021.00,General and Operations Managers,1
3,11-2011.00,Advertising and Promotions Managers,1
4,11-2021.00,Marketing Managers,1
...,...,...,...
963,53-7072.00,"Pump Operators, Except Wellhead Pumpers",0
964,53-7073.00,Wellhead Pumpers,0
965,53-7081.00,Refuse and Recyclable Material Collectors,0
966,53-7111.00,Mine Shuttle Car Operators,0


### Tradução do O\*NET (filtrado por Dingel e Neiman) para o SOC-2010 

A tradução deve ser feita usando como refência o código do Major Group e título da ocupação

In [117]:
soc2010filtrado = soc2010[['Detailed Occupation', 'Title']]
soc2010filtrado.columns = ['code_SOC', 'Title_SOC']
soc2010filtrado.dropna(inplace=True)

soc2010['code_SOC'] = soc2010.iloc[:, :4].apply(lambda x: ''.join(x.dropna().astype(str)), axis=1)
#Captura somente o codigo major
soc2010filtrado['MAJOR_SOC'] = soc2010filtrado['code_SOC'].apply(lambda x: x[:2])

C:\Users\est.matheus.brasil\AppData\Local\Temp\ipykernel_12948\1001432072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soc2010filtrado.dropna(inplace=True)
C:\Users\est.matheus.brasil\AppData\Local\Temp\ipykernel_12948\1001432072.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soc2010filtrado['MAJOR_SOC'] = soc2010filtrado['code_SOC'].apply(lambda x: x[:2])


In [118]:
onet_filtrado = onet_teleworkable.copy()
onet_filtrado.columns = ['code_ONET', 'Title_ONET', 'teleworkable']

#captura somente o codigo major
onet_filtrado['code_ONET'] = onet_filtrado['code_ONET'].apply(lambda x: x[0:-3])

In [119]:
#join onet_filtrado e soc2010filtrado by column Code_ONET and code_SOC

ocupacoes_agregado = pd.merge(onet_filtrado, soc2010filtrado, how='left', left_on='code_ONET', right_on='code_SOC')

In [120]:
ocupacoes_agregado.head(10)

,code_ONET,Title_ONET,teleworkable,code_SOC,Title_SOC,MAJOR_SOC
0,11-1011,Chief Executives,1,11-1011,Chief Executives,11
1,11-1011,Chief Sustainability Officers,1,11-1011,Chief Executives,11
2,11-1021,General and Operations Managers,1,11-1021,General and Operations Managers,11
3,11-2011,Advertising and Promotions Managers,1,11-2011,Advertising and Promotions Managers,11
4,11-2021,Marketing Managers,1,11-2021,Marketing Managers,11
5,11-2022,Sales Managers,1,11-2022,Sales Managers,11
6,11-2031,Public Relations and Fundraising Managers,1,11-2031,Public Relations and Fundraising Managers,11
7,11-3011,Administrative Services Managers,1,11-3011,Administrative Services Managers,11
8,11-3021,Computer and Information Systems Managers,1,11-3021,Computer and Information Systems Managers,11
9,11-3031,Treasurers and Controllers,1,11-3031,Financial Managers,11


Até aqui, ONET e SOC-2010 foram agregados. Agora, vamos fazer o mesmo com ISCO-08 e COD-2010.

### Tradução de ISC08 para COD 2010

In [131]:
cod2010_filtrado = cod2010[[0, 'Titulação']]
cod2010_filtrado.columns = ['code_COD', 'Title_COD']

#Captura somente o codigo major
#cod2010_filtrado['MAJOR_cod'] = soc2010filtrado['code_SOC'].apply(lambda x: x[:2])

cod2010_filtrado['code_COD'] = cod2010_filtrado['code_COD'].astype(str)

C:\Users\est.matheus.brasil\AppData\Local\Temp\ipykernel_12948\2344969089.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cod2010_filtrado['code_COD'] = cod2010_filtrado['code_COD'].astype(str)


In [132]:
isco08_filtrado = isco08[['ISCO 08 Code', 'Title EN']]
isco08_filtrado.columns = ['code_ISCO', 'Title_ISCO']

isco08_filtrado['code_ISCO'] = isco08_filtrado['code_ISCO'].astype(str) 

C:\Users\est.matheus.brasil\AppData\Local\Temp\ipykernel_12948\561784860.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isco08_filtrado['code_ISCO'] = isco08_filtrado['code_ISCO'].astype(str)


In [133]:
isco_cod = pd.merge(isco08_filtrado, cod2010_filtrado, how='left', left_on='code_ISCO', right_on='code_COD')

Até aqui, temos ISCO-08 e COD 2010 agregados. Agora, vamos fazer o join com as ocupações agregadas.

### Tradução de SOC-2010 para ISCO-08

In [136]:
ocupacoes_agregado = pd.merge(ocupacoes_agregado, isco_cod, how='left', left_on='MAJOR_SOC', right_on='code_ISCO')

In [140]:
ocupacoes_agregado.fillna('OUTROS', inplace=True)

#### trabalhos teletrabalháveis

In [141]:
ocupacoes_agregado.groupby('Title_COD')['code_COD'].count()

Title_COD
DIRETORES EXECUTIVOS, DIRIGENTES DA ADMINISTRAÇÃO PÚBLICA E MEMBROS DO PODER EXECUTIVO E LEGISLATIVO    112
DIRIGENTES E GERENTES DE PRODUÇÃO E OPERAÇÃO                                                             49
OUTROS                                                                                                  762
PROFISSIONAIS DAS CIÊNCIAS E DA ENGENHARIA                                                               28
PROFISSIONAIS DE NÍVEL MÉDIO DAS CIÊNCIAS E DA ENGENHARIA                                                36
PROFISSIONAIS DE TECNOLOGIAS DA INFORMAÇÃO E COMUNICAÇÕES                                                61
PROFISSIONAIS DO ENSINO                                                                                   8
Name: code_COD, dtype: int64

In [145]:
ocupacoes_agregado[ocupacoes_agregado['teleworkable'] == 1].groupby('Title_ISCO')['code_COD'].count() / ocupacoes_agregado.groupby('Title_COD')['code_COD'].count()

Armed Forces Occupations, Other Ranks                                                                        NaN
Business and Administration Associate Professionals                                                          NaN
Chief Executives, Senior Officials and Legislators                                                           NaN
Commissioned Armed Forces Officers                                                                           NaN
DIRETORES EXECUTIVOS, DIRIGENTES DA ADMINISTRAÇÃO PÚBLICA E MEMBROS DO PODER EXECUTIVO E LEGISLATIVO         NaN
DIRIGENTES E GERENTES DE PRODUÇÃO E OPERAÇÃO                                                                 NaN
General and Keyboard Clerks                                                                                  NaN
Information and Communications Technicians                                                                   NaN
Information and Communications Technology Professionals                                         

In [148]:
soc2010

,Major Group,Minor Group,Broad Group,Detailed Occupation,Title,code_SOC
0,11-0000,NaN,NaN,NaN,Management Occupations,11-0000
1,NaN,11-1000,NaN,NaN,Top Executives,11-1000
2,NaN,NaN,11-1010,NaN,Chief Executives,11-1010
3,NaN,NaN,NaN,11-1011,Chief Executives,11-1011
4,NaN,NaN,11-1020,NaN,General and Operations Managers,11-1020
...,...,...,...,...,...,...
1416,NaN,NaN,NaN,55-3015,Command and Control Center Specialists,55-3015
1417,NaN,NaN,NaN,55-3016,Infantry,55-3016
1418,NaN,NaN,NaN,55-3017,Radar and Sonar Technicians,55-3017
1419,NaN,NaN,NaN,55-3018,Special Forces,55-3018


In [147]:
isco_cod

,code_ISCO,Title_ISCO,code_COD,Title_COD
0,1,Managers,1,DIRETORES E GERENTES
1,11,"Chief Executives, Senior Officials and Legisla...",11,"DIRETORES EXECUTIVOS, DIRIGENTES DA ADMINISTRA..."
2,111,Legislators and Senior Officials,111,MEMBROS SUPERIORES DO PODER EXECUTIVO E LEGISL...
3,1111,Legislators,1111,LEGISLADORES
4,1112,Senior Government Officials,1112,DIRIGENTES SUPERIORES DA ADMINISTRAÇÃO PÚBLICA
...,...,...,...,...
614,21,Non-commissioned Armed Forces Officers,21,PROFISSIONAIS DAS CIÊNCIAS E DA ENGENHARIA
615,210,Non-commissioned Armed Forces Officers,NaN,NaN
616,3,"Armed Forces Occupations, Other Ranks",3,TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO
617,31,"Armed Forces Occupations, Other Ranks",31,PROFISSIONAIS DE NÍVEL MÉDIO DAS CIÊNCIAS E DA...
